: 

In [6]:
import sys
from pathlib import Path
import json

import pandas as pd
import numpy as np
import geopandas as gp


import gtfs_kit as gk


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
gtfs_path = Path("data")

gk.list_feed(gtfs_path)

,file_name,file_size
0,api.txt,2274702
1,trips.txt,2766590
2,transfers.txt,6367503
3,stops.txt,809861
4,stop_times.txt,36256066
5,routes.txt,27856
6,calendar.txt,2279
7,calendar_dates.txt,558
8,agency.txt,174


In [9]:
feed = gk.read_feed(gtfs_path, dist_units='km')
feed.describe()

,indicator,value
0,agencies,"[IDS JMK (Data from: KORDIS JMK, DPMB)]"
1,timezone,Europe/Prague
2,start_date,20241213
3,end_date,20241224
4,num_routes,341
5,num_trips,56223
6,num_stops,10832
7,num_shapes,0
8,sample_date,20241219
9,num_routes_active_on_sample_date,332


In [28]:
stops_df = gk.get_stops(feed)
stops_df[stops_df["stop_name"].str.contains("Šumavská")]


,stop_id,stop_name,stop_lat,stop_lon,zone_id,location_type,parent_station,wheelchair_boarding,platform_code
6489,U1652Z1,Šumavská,49.213991,16.600669,100,0,U1652N2846,1,NaN
6507,U1652Z2,Šumavská,49.214022,16.600620,100,0,U1652N2846,1,NaN
10438,U1652N2846,Šumavská,49.214007,16.600645,100,1,<NA>,1,NaN


In [46]:
from datetime import datetime

time_now = datetime.now().time().strftime("%H:%M:%S")
date_today = datetime.now().date().strftime("%Y%m%d")

print(time_now)
print(date_today)

timetable_df = gk.build_stop_timetable(feed, "U1652Z2", [date_today])
current_departures = timetable_df[timetable_df["departure_time"] > time_now]
timetable_df.dtypes


11:31:01
20241217


route_id                 string[python]
service_id               string[python]
trip_id                  string[python]
trip_headsign            string[python]
wheelchair_accessible              Int8
block_id                 string[python]
direction_id                       Int8
bikes_allowed                      Int8
exceptional                       int64
arrival_time             string[python]
departure_time           string[python]
stop_id                  string[python]
stop_sequence                     Int32
pickup_type                        Int8
drop_off_type                      Int8
date                             object
dtype: object

In [52]:
timetable_df[pd.to_timedelta(timetable_df["departure_time"], errors="coerce") > time_now]


,route_id,service_id,trip_id,trip_headsign,wheelchair_accessible,block_id,direction_id,bikes_allowed,exceptional,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,date
152,L1D99,1,32026,Ečerova,1,<NA>,0,1,0,11:32:00,11:32:00,U1652Z2,10,0,0,20241217
153,L6D99,1,33558,"Starý Lískovec, smyčka",2,<NA>,0,1,0,11:35:00,11:35:00,U1652Z2,6,0,0,20241217
154,L1D99,1,32041,Ečerova,1,<NA>,0,1,0,11:38:00,11:38:00,U1652Z2,10,0,0,20241217
155,L6D99,1,33575,"Starý Lískovec, smyčka",1,<NA>,0,1,0,11:42:00,11:42:00,U1652Z2,6,0,0,20241217
156,L1D99,1,32056,Ečerova,1,<NA>,0,1,0,11:45:00,11:45:00,U1652Z2,10,0,0,20241217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,L90D99,58,55152,"Kohoutovice, hájenka",1,<NA>,0,1,0,23:17:00,23:17:00,U1652Z2,21,3,3,20241217
334,L91D99,58,55548,Labská,1,<NA>,1,1,0,23:17:00,23:17:00,U1652Z2,17,3,3,20241217
335,L90D99,58,55167,"Kohoutovice, hájenka",1,<NA>,0,1,0,23:47:00,23:47:00,U1652Z2,21,3,3,20241217
336,L91D99,58,55203,Labská,1,<NA>,1,1,0,23:47:00,23:47:00,U1652Z2,24,3,3,20241217


In [14]:
trips_df = gk.get_trips(feed)
trips_df

,route_id,service_id,trip_id,trip_headsign,wheelchair_accessible,block_id,direction_id,bikes_allowed,exceptional
0,L104D99,1,1,ÚAN Zvonařka,1,<NA>,1,0,0
1,L104D99,1,2,"Laa an der Thaya, Therme",1,<NA>,0,0,0
2,L104D99,1,3,"Pohořelice, náměstí",1,<NA>,1,0,0
3,L104D99,1,4,"Laa an der Thaya, Therme",1,<NA>,0,0,0
4,L104D99,1,5,ÚAN Zvonařka,1,<NA>,1,0,0
...,...,...,...,...,...,...,...,...,...
56218,L99D99,61,56219,Jírova,1,<NA>,0,1,0
56219,L99D99,61,56220,Technologický park,1,<NA>,1,1,0
56220,L44D99,61,56221,ÚAN Zvonařka,1,<NA>,0,1,0
56221,L99D99,61,56222,Technologický park,1,<NA>,1,1,0


In [19]:
routes_df = gk.get_routes(feed)
routes_df[(routes_df["route_short_name"] == "6") | (routes_df["route_short_name"] == "1")]


,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color
55,L1D99,99,1,Řečkovice - Pisárky,0,D40000,FFFFFF
248,L6D99,99,6,"Královo Pole, nádraží - Starý Lískovec, smyčka",0,0777C1,FFFFFF


In [31]:
routes_df[routes_df["route_id"] == "L90D99"]

,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color
318,L90D99,99,N90,"Ořešín - Hlavní nádraží - Kohoutovice, hájenka",3,000000,E1CB31


In [63]:
from timetable import get_timetables
import logging

logging.basicConfig(level=logging.INFO)

feed = gk.read_feed(gtfs_path, dist_units='km')
timetable = get_timetables(feed, "Šumavská")
for timetable in timetable:
    print(timetable)




[autoreload of timetable failed: Traceback (most recent call last):
  File "/home/pchmelar/repos/timetable/.venv/lib64/python3.13/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
    ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/pchmelar/repos/timetable/.venv/lib64/python3.13/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
    ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^
  File "/home/pchmelar/repos/timetable/.venv/lib64/python3.13/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
    ~~~~~~^^^^^^
  File "/home/pchmelar/repos/timetable/.venv/lib64/python3.13/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
       ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^
  File "/home/pchmelar/repos/timetable/.venv/lib64/python3.13/site-packages/IPython/extensions/autoreload.py", 

Šumavská
1 - Řečkovice - 12:06:00
Šumavská
6 - Královo Pole, nádraží - 12:11:00
Šumavská
1 - Řečkovice - 12:13:00
Šumavská
6 - Královo Pole, nádraží - 12:18:00
Šumavská
1 - Řečkovice - 12:20:00
Šumavská
6 - Královo Pole, nádraží - 12:24:00
Šumavská
1 - Řečkovice - 12:26:00
Šumavská
6 - Královo Pole, nádraží - 12:31:00
Šumavská
1 - Řečkovice - 12:33:00
Šumavská
6 - Královo Pole, nádraží - 12:38:00
Šumavská
6 - Starý Lískovec, smyčka - 12:09:00
Šumavská
1 - Ečerova - 12:12:00
Šumavská
6 - Starý Lískovec, smyčka - 12:15:00
Šumavská
1 - Ečerova - 12:18:00
Šumavská
6 - Starý Lískovec, smyčka - 12:22:00
Šumavská
1 - Ečerova - 12:25:00
Šumavská
6 - Starý Lískovec, smyčka - 12:29:00
Šumavská
1 - Ečerova - 12:32:00
Šumavská
6 - Starý Lískovec, smyčka - 12:35:00
Šumavská
1 - Ečerova - 12:38:00


In [3]:
import requests

url = "https://www.arcgis.com/sharing/rest/content/items/379d2e9a7907460c8ca7fda1f3e84328/data"

r = requests.head(url)
print(r.headers)


{'Date': 'Sat, 21 Dec 2024 14:56:18 GMT', 'Content-Type': 'application/zip', 'Content-Length': '7856326', 'Connection': 'keep-alive', 'Vary': 'Origin', 'X-XSS-Protection': '1; mode=block', 'X-Frame-Options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff', 'Strict-Transport-Security': 'max-age=31536000', 'Content-Disposition': 'attachment; filename="GTFS.zip"; size=7856326', 'Accept-Ranges': 'bytes'}
